In [48]:
import requests
import math
import io
import imutils
import os
import cv2
from PIL import ImageOps, Image, ImageDraw, ImageFont
from itertools import chain

def slope(x, y):
    sl = math.sqrt(x**2 + y**2)
    return sl

def get_ann(img_path,api_url) -> dict:
    ''' img_path : str or PIL image '''
    if type(img_path) == str:
        file_dict = {"file": open(img_path  , "rb")}
    else:
        output = io.BytesIO()
        image = img_path
        image.save(output, format="JPEG")
        file_dict = {"file": output.getvalue()}
    headers = {"secret": "Boostcamp0001"}
    response = requests.post(api_url, headers=headers, files=file_dict)
    return response.json()


def get_degree(annos):

    # 제일 대표적인 bbox를 찾음
    horizontal_list = []

    for idx, anno in enumerate(annos):
        xlen = anno['points'][1][0] - anno['points'][0][0] # x축 길이 차
        ylen = anno['points'][0][1] - anno['points'][1][1] # y축 길이 차
        ylen = abs(ylen)
        horizontal_list.append((slope(xlen,ylen), idx))

    longest = max(horizontal_list)[1]
    # print(annos[longest])

    # 각도 계산
    thetaplus = False
    xlen = annos[longest]['points'][1][0] - annos[longest]['points'][0][0]
    ylen = annos[longest]['points'][0][1] - annos[longest]['points'][1][1] # 음수일 수도 있음

    if ylen < 0 :
        thetaplus = True
        ylen = abs(ylen)

    costheta = xlen / slope(xlen, ylen)
    theta = math.acos(costheta)
    degree = round(theta * 57.29,3)

    if thetaplus == True:
        degree = degree
    else:
        degree = -degree

    return degree

In [49]:
folder_path = '/opt/ml/upstage_OCR/Data set/real data/receipt'
imagelist = sorted(os.listdir(folder_path))
for path in imagelist:
    imgpath = os.path.join(folder_path, path)
    print(path)
    ann_dict = get_ann(imgpath, "http://118.222.179.32:30001/ocr/")
    annos = ann_dict['ocr']['word']
    degree = get_degree(annos)
    image = cv2.imread(imgpath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  
    rotated = imutils.rotate_bound(image, -degree)
    pil_rotate=Image.fromarray(rotated)
    ann_dict2 = get_ann(pil_rotate,"http://118.222.179.32:30001/ocr/")
    annos2 = ann_dict2['ocr']['word']

    texts = []
    for ind, anno in enumerate(annos2):
        texts.append((ind, anno['points'][0],anno['points'][1], anno['text'], anno['points'][2],anno['points'][3],)) 
    
    texts_ = sorted(texts, key = lambda x: (x[1][1],x[1][0]))  # y로 정렬 후 x정렬

    tmp = texts_[0][1][1] # 첫번째 글자의 y좌표 
    align = []
    phase = []
    for text in texts_:
        new_phase = []
        if abs(text[1][1] - tmp) <= ((text[5][1] - text[1][1])/1.5) :  #  같은 라인 판별 / 글자의 반 이내면 
            phase.append(text)
            tmp = min(text[2][1], text[1][1])
        else: # tmp값 벌어지면 다음 라인 취급
            phase.sort(key = lambda x: x[1][0]) # x로 정렬
            align.append(phase)
            new_phase.append(text)
            phase = new_phase
            tmp = min(text[2][1], text[1][1])

    phase.sort(key = lambda x: x[1][0])
    align.append(phase) # 마지막 줄 추가

    # for i in align:
    #     for n in i:
    #         print(n[3] ,end="  ")
    #     print(" ") 

    line = []
    word = []
    for i in align:
        tmp = i[0][1][0]
        for n in i:
            if n[1][0] - tmp <= ((n[2][0]-n[1][0])/len(n[3]))/1.5: 
                word.append(n[3])
                tmp = n[2][0]
            else:
                word.append(" ")
                word.append(n[3])
                tmp = n[2][0]
        line.append(word)
        word = []

    for i in line:
        s = "".join(i)
        print(s)

    print("\n=========================================================================================")
        

receipt000.jpg
출입확인증   
Jabook  Work  &  Study  카페   
20시간권  좌석  :  29번(  7340)   
*  3  3  6  1  1  *   
총남은시간:  9시간17분   
시작:  2022-04-30  15:02:28   
종료:  2022-05-01  00:20:08   
전화  :010-  3322-2770  화장  실pw  :5070*   
id  :  1  abook  A5G.  pw  :  1  abook2770   
id  :  ]  abook  A2  L  4G  .  pw  :  1  abook2770   

receipt001.jpg
네이버   
카페인앓O  |  인플루언서   
영수증NO:0030-1   
사업자번호  :7872100626   
주소  :부산  연제구  중앙대로1043번길  12  1층   
성명  :강창욱   
전화  :  01087254967   
일자  :  2020-08-05  14:44:55   
품명  단가  수량  금액   
겨울왕국무유  2,500  2  5,000   
골드메달  애플쥬스  (무탄산)   
3,500  1  3,500   
그린티라떼  ★  ICE  4,500  1  4,500   
딸기요거트스무디  4,500  1  4,500   
쏠티카라멜  2,500  1  2,500   
카라멜  마키아또  ★  ICE   
5,000  1  5,000   
소  계:  25,000   
품명  앞에  *  표시가  되어있는  품목은   
부가세  면세  품목입니다.   
부가세  과세  물품가액:  22,727   
부  가  세:  2,273   
부가세  면세  물품가액:  0   
청구금액:  25,000   
받은금액:  25,000   
거스름돈:  0   
현  금:  25,000   
Wifi  Name  C  afe  _  in_RE-5G   
Wifi  Passwor  d  -  r  e-  180731   
결제취소시  반드시  영수증